# Laboratorio 3 Base de Datos 2

### Juan M. González-Campo 21077
### Paulo R. Sánchez 21401
### Pedro J. Marroquín 21801

In [26]:

#Importacion de librerias
from neo4j import GraphDatabase
#Datos de conexión
URI="neo4j+ssc://f955a66e.databases.neo4j.io"
AUTH=("neo4j","9h4UWCWtkxjqM90NQrXDmDmF8O342uMJrrpRsyP58mU")
AURA_INSTANCEID="f955a66e"
AURA_INSTANCENAME="Instance01"
#Conexión

## Prueba de Conexión

In [27]:
driver = GraphDatabase.driver(URI, auth=AUTH)

# Test the connection with a simple query
def test_connection(driver):
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful' AS message")
        print(result.single()["message"])


test_connection(driver)


Connection successful


# Ejercicio 1

#### En el lenguaje de programación de su preferencia, implementar funciones para crear el grafo (la idea es que el grafo se cree a partir de funciones que reciban parámetros)


In [28]:
# 1. En el lenguaje de programación de su preferencia, implementar funciones para crear el grafo (la idea es que el grafo se cree a partir de funciones que reciban parámetros)
#Crea un usuario
def create_user(driver,name,userId):
    result=driver.execute_query(
        """
        MERGE (u:User{name:$name, userId:$userId}) 
        RETURN u
        """,
        name=name,userId=userId,database="neo4j"
    )
    print(f"Usuario '{name}' (ID: {userId}) creado.")
    return result
#Crea una pelicula
def create_movie(driver,title,movieId,year,plot):
    result=driver.execute_query(
    """
    MERGE (m:Movie{title:$title, movieId:$movieId, year:$year, plot:$plot})
    RETURN m
    """,
    title=title, movieId=movieId, year=year, plot=plot, database="neo4j"
    )
    print(f"Película '{title}' (ID: {movieId}) creada.")
    return result

#Crea una conexión entre un usuario y una película
def user_rates_movie(driver,userId,movieId,rating,timestamp):
    if timestamp == None:
        result=driver.execute_query(
            """
            MATCH (u:User {userId:$userId})
            MATCH (m:Movie {movieId:$movieId})
            MERGE (u)-[r:RATED {rating:$rating, timestamp: timestamp()}]->(m)
            RETURN u, r, m
            """,
            userId=userId, movieId=movieId, rating=rating, database="neo4j"
        )
    else:
        result=driver.execute_query(
            """
            MATCH (u:User {userId:$userId})
            MATCH (m:Movie {movieId:$movieId})
            MERGE (u)-[r:RATED {rating:$rating, timestamp:$timestamp}]->(m)
            RETURN u, r, m
            """,
            userId=userId, movieId=movieId, rating=rating, timestamp=timestamp, database="neo4j"
        )
    print(f"Usuario '{userId}' ha calificado la película '{movieId}' con {rating} estrellas.")
    return result

#### 2. Popular el grafo con 5 registros (5 usuarios que tengan un rate hacia como mínimo dos películas distintas).

In [29]:
# 2. Popular el grafo con 5 registros (5 usuarios que tengan un rate hacia como mínimo dos películas distintas).
# Datos de películas
peliculas = [
    {"title": "Inception", "movieId": 101, "year": 2010, "plot": "A thief who steals corporate secrets through the use of dream-sharing technology."},
    {"title": "The Matrix", "movieId": 102, "year": 1999, "plot": "A computer hacker learns from mysterious rebels about the true nature of his reality."}
]

# Usuarios
usuarios= [
    {"name": "Alice", "userId": 1},
    {"name": "Bob", "userId": 2},
    {"name": "Charlie", "userId": 3},
    {"name": "David", "userId": 4},
    {"name": "Eve", "userId": 5}
    
]

# Relaciones

relaciones = [
    {"userId": 1, "movieId": 101, "rating": 5, "timestamp": None},
    {"userId": 1, "movieId": 102, "rating": 4, "timestamp": None},
    {"userId": 2, "movieId": 101, "rating": 3, "timestamp": None},
    {"userId": 2, "movieId": 102, "rating": 5, "timestamp": None},
    {"userId": 3, "movieId": 101, "rating": 4, "timestamp": None},
    {"userId": 3, "movieId": 102, "rating": 3, "timestamp": None},
    {"userId": 4, "movieId": 101, "rating": 5, "timestamp": None},
    {"userId": 4, "movieId": 102, "rating": 5, "timestamp": None},
    {"userId": 5, "movieId": 101, "rating": 4, "timestamp": None},
    {"userId": 5, "movieId": 102, "rating": 3, "timestamp": None}
]

for i in peliculas:
    create_movie(driver, i["title"], i["movieId"], i["year"], i["plot"])

for i in usuarios:
    create_user(driver, i["name"], i["userId"])

for i in relaciones:
    user_rates_movie(driver, i["userId"], i["movieId"], i["rating"], i["timestamp"])


Película 'Inception' (ID: 101) creada.
Película 'The Matrix' (ID: 102) creada.
Usuario 'Alice' (ID: 1) creado.
Usuario 'Bob' (ID: 2) creado.
Usuario 'Charlie' (ID: 3) creado.
Usuario 'David' (ID: 4) creado.
Usuario 'Eve' (ID: 5) creado.
Usuario '1' ha calificado la película '101' con 5 estrellas.
Usuario '1' ha calificado la película '102' con 4 estrellas.
Usuario '2' ha calificado la película '101' con 3 estrellas.
Usuario '2' ha calificado la película '102' con 5 estrellas.
Usuario '3' ha calificado la película '101' con 4 estrellas.
Usuario '3' ha calificado la película '102' con 3 estrellas.
Usuario '4' ha calificado la película '101' con 5 estrellas.
Usuario '4' ha calificado la película '102' con 5 estrellas.
Usuario '5' ha calificado la película '101' con 4 estrellas.
Usuario '5' ha calificado la película '102' con 3 estrellas.


In [30]:

# 3. Implementar función para encontrar un usuario, una película y un usuario con su relación rate a película
#Encontrar usuario por Id
def find_user_by_id(driver, userId):
    result = driver.execute_query(
        """
        MATCH (u:User {userId: $userId})
        RETURN u
        """,
        userId=userId, database="neo4j"
    )
    if result.records:
        print(f"Usuario encontrado: {result.records[0]['u']}")
    else:
        print(f"No se encontró ningún usuario con ID: {userId}")
    return result

#Encontrar película por ID
def find_movie_by_id(driver, movieId):
    result = driver.execute_query(
        """
        MATCH (m:Movie {movieId: $movieId})
        RETURN m
        """,
        movieId=movieId, database="neo4j"
    )
    if result.records:
        print(f"Película encontrada: {result.records[0]['m']}")
    else:
        print(f"No se encontró ninguna película con ID: {movieId}")
    return result


#Encontrar la calificación de un usuario para una película
def find_user_movie_rating(driver, userId, movieId):
    result = driver.execute_query(
        """
        MATCH (u:User {userId: $userId})-[r:RATED]->(m:Movie {movieId: $movieId})
        RETURN u, r, m
        """,
        userId=userId, movieId=movieId, database="neo4j"
    )

    # Verificar si hay resultados
    if result.records:  
        record = result.records[0]  
        user = record["u"]
        movie = record["m"]
        rating = record["r"]

        return {
            "User": {"id": user["userId"], "name": user["name"]},
            "Movie": {"id": movie["movieId"], "title": movie["title"], "year": movie["year"]},
            "Rating": {"rating": rating["rating"], "timestamp": rating["timestamp"]}
        }
    else:
        return f"No se encontro un rating del usuario {userId} hacia la pelicula {movieId}."

In [31]:
u = find_user_by_id(driver, 1)

m = find_movie_by_id(driver, 101)

umr = find_user_movie_rating(driver, 1, 101)
print(umr)



Usuario encontrado: <Node element_id='4:41c33069-6546-4412-a791-1b857f5a5a3d:6' labels=frozenset({'User'}) properties={'name': 'Alice', 'userId': 1}>
Película encontrada: <Node element_id='4:41c33069-6546-4412-a791-1b857f5a5a3d:4' labels=frozenset({'Movie'}) properties={'year': 2010, 'plot': 'A thief who steals corporate secrets through the use of dream-sharing technology.', 'movieId': 101, 'title': 'Inception'}>
{'User': {'id': 1, 'name': 'Alice'}, 'Movie': {'id': 101, 'title': 'Inception', 'year': 2010}, 'Rating': {'rating': 5, 'timestamp': 1739142943188}}


#### 4. Ahora, con las funciones previamente creadas, deberá generar el siguiente grafo. Es importante que tome en cuenta todas las propiedades de cada label y de aquellas relaciones que tengan alguna

In [32]:
# Definimos nuevas funciones necesarias

# Crear pelicula con más parámetros
def create_movie_more_params(driver,title,tmdbId,released,imdbRating,movieId,year,imdbId,runtime,countries,imdbVotes,url,revenue,plot,poster,budget,languages):
    driver.execute_query(
        """
        MERGE (m:Movie{title:$title,tmdbId:$tmdbId,released:$released,imdbRating:$imdbRating,movieId:$movieId,year:$year,imdbId:$imdbId,runtime:$runtime,countries:$countries,imdbVotes:$imdbVotes,url:$url,revenue:$revenue,plot:$plot,poster:$poster,budget:$budget,languages:$languages})
        RETURN m
        """,
        title=title, tmdbId=tmdbId, released=released, imdbRating=imdbRating, 
        movieId=movieId, year=year, imdbId=imdbId, runtime=runtime, countries=countries,
        imdbVotes=imdbVotes, url=url, revenue=revenue, plot=plot, poster=poster, budget=budget, 
        languages=languages, database="neo4j"

    )
    print(f"Película '{title}' (ID: {movieId}) creada.")

# Crear personas que pueden tener subtipos
def create_person(driver, subtype, name, tmdbId, born, died, bornIn, url, imdbId, bio, poster):
    if subtype == 'Actor':
        driver.execute_query(
            """
            MERGE (p:Person:Actor {name:$name,tmdbId:$tmdbId,born:$born,died:$died,bornIn:$bornIn,url:$url,imdbId:$imdbId,bio:$bio,poster:$poster})
            RETURN p
            """,
            name=name, tmdbId=tmdbId, born=born, died=died, bornIn=bornIn, url=url, imdbId=imdbId, bio=bio, poster=poster, database="neo4j"
        )
    elif subtype == 'Director':
         driver.execute_query(
            """
            MERGE (p:Person:Director {name:$name, tmdbId:$tmdbId, born:$born,died:$died,bornIn:$bornIn,url:$url,imdbId:$imdbId,bio:$bio,poster:$poster})
            RETURN p
            """,
            name=name, tmdbId=tmdbId, born=born, died=died, bornIn=bornIn, url=url, imdbId=imdbId, bio=bio, poster=poster, database="neo4j"
        )
    elif subtype == 'Actor:Director':
        driver.execute_query(
            """
            MERGE (p:Person:Actor:Director {name:$name,tmdbId:$tmdbId,born:$born,died:$died,bornIn:$bornIn,url:$url,imdbId:$imdbId,bio:$bio,poster:$poster})
            RETURN p
            """,
            name=name, tmdbId=tmdbId, born=born, died=died, bornIn=bornIn, url=url, imdbId=imdbId, bio=bio, poster=poster, database="neo4j"
        )
    else:
        driver.execute_query(
            """
            MERGE (p:Person {name:$name,tmdbId:$tmdbId,born:$born,died:$died,bornIn:$bornIn,url:$url,imdbId:$imdbId,bio:$bio,poster:$poster})
            RETURN p
            """,
            name=name, tmdbId=tmdbId, born=born, died=died, bornIn=bornIn, url=url, imdbId=imdbId, bio=bio, poster=poster, database="neo4j"
        )
    print(f"Persona '{name}' creada.")

# Crear género
def create_genre(driver, name):
    driver.execute_query(
        """
        MERGE (g:Genre{name:$name})
        RETURN g
        """,
        name=name, database="neo4j"
    )
    print(f"Género '{name}' creado.")

# Crear relación entre Director y película
def link_director(driver, personName, movieTitle):
    driver.execute_query(
        """
        MATCH (p:Person {name:$personName})
        MATCH (m:Movie {title:$movieTitle})
        MERGE (p)-[:DIRECTED]->(m)
        RETURN p,m
        """,
        personName=personName, movieTitle=movieTitle, database="neo4j"
    )
    print(f"Relación creada entre '{personName}' y '{movieTitle}'.")

#Crear relación entre Actor y película
def link_actor(driver, personName, movieTitle):
    driver.execute_query(
        """
        MATCH (p:Person {name:$personName})
        MATCH (m:Movie {title:$movieTitle})
        MERGE (p)-[:ACTED_IN]->(m)
        RETURN p,m
        """,
        personName=personName, movieTitle=movieTitle, database="neo4j"
    )
    print(f"Relación creada entre '{personName}' y '{movieTitle}'.")

# Crear relación entre Género y película
def link_genre(driver, genreName, movieTitle):
    driver.execute_query(
        """
        MATCH (g:Genre {name:$genreName})
        MATCH (m:Movie {title:$movieTitle})
        MERGE (g)-[:IN_GENRE]->(m)
        RETURN g,m
        """,
        genreName=genreName, movieTitle=movieTitle, database="neo4j"
    )
    print(f"Relación creada entre '{genreName}' y '{movieTitle}'.")
    


In [33]:
create_movie_more_params(driver, "The Dark Knight", 155, "2008-07-18", 9.0, 103, 2008, "tt0468569", 152, ["USA", "UK"], 2000000, "https://www.imdb.com/title/tt0468569/", 1000000000, "When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.", "https://m.media-amazon.com/images/I/81v90Jf4G-L._AC_SY679_.jpg", 185000000, ["English", "Mandarin", "Arabic"])

create_person(driver, "Director", "Christopher Nolan", 525, "1970-07-30", "2025-01-01", "London, England, UK", "https://www.imdb.com/name/nm0634240/?ref_=tt_ov_dr_1", "nm0634240", "Best known for his cerebral, often nonlinear storytelling, acclaimed writer-director Christopher Nolan was born on July 30, 1970 in London, England.", "https://www.imdb.com/name/nm0634240/mediaviewer/rm2047771392/?ref_=nm_ov_ph")

create_person(driver, "Actor", "Christian Bale", 3894, "1974-01-30", "2025-01-01", "Haverfordwest, Pembrokeshire, Wales, UK", "https://www.imdb.com/name/nm0000288/?ref_=tt_ov_st_sm", "nm0000288", "Christian Charles Philip Bale was born in Pembrokeshire, Wales, UK on January 30, 1974, to English parents Jennifer Jenny (James) and David Bale.", "https://www.imdb.com/name/nm0000288/mediaviewer/rm3114052352/?ref_=nm_ov_ph")

create_genre(driver, "Action")

create_person(driver, "Actor:Director", "Heath Ledger", 51329, "1979-04-04", "2008-01-22", "Perth, Western Australia, Australia", "https://www.imdb.com/name/nm0005132/?ref_=tt_ov_st_sm", "nm0005132", "When hunky, twenty-year-old heart-throb Heath Ledger first came to the attention of the public in 1999, it was all too easy to tag him as a pretty.", "https://www.imdb.com/name/nm0005132/mediaviewer/rm3114052352/?ref_=nm_ov_ph")

link_director(driver, "Christopher Nolan", "The Dark Knight")

link_actor(driver, "Christian Bale", "The Dark Knight")

link_actor(driver, "Heath Ledger", "The Dark Knight")

link_director(driver, "Heath Ledger", "The Dark Knight")

link_genre(driver, "Action", "The Dark Knight")

user_rates_movie(driver, 1, 103, 5, None)


Película 'The Dark Knight' (ID: 103) creada.
Persona 'Christopher Nolan' creada.
Persona 'Christian Bale' creada.
Género 'Action' creado.
Persona 'Heath Ledger' creada.
Relación creada entre 'Christopher Nolan' y 'The Dark Knight'.
Relación creada entre 'Christian Bale' y 'The Dark Knight'.
Relación creada entre 'Heath Ledger' y 'The Dark Knight'.
Relación creada entre 'Heath Ledger' y 'The Dark Knight'.
Relación creada entre 'Action' y 'The Dark Knight'.
Usuario '1' ha calificado la película '103' con 5 estrellas.


EagerResult(records=[<Record u=<Node element_id='4:41c33069-6546-4412-a791-1b857f5a5a3d:6' labels=frozenset({'User'}) properties={'name': 'Alice', 'userId': 1}> r=<Relationship element_id='5:41c33069-6546-4412-a791-1b857f5a5a3d:1157425104234217478' nodes=(<Node element_id='4:41c33069-6546-4412-a791-1b857f5a5a3d:6' labels=frozenset({'User'}) properties={'name': 'Alice', 'userId': 1}>, <Node element_id='4:41c33069-6546-4412-a791-1b857f5a5a3d:12' labels=frozenset({'Movie'}) properties={'languages': ['English', 'Mandarin', 'Arabic'], 'year': 2008, 'imdbId': 'tt0468569', 'runtime': 152, 'movieId': 103, 'imdbRating': 9.0, 'countries': ['USA', 'UK'], 'title': 'The Dark Knight', 'imdbVotes': 2000000, 'url': 'https://www.imdb.com/title/tt0468569/', 'revenue': 1000000000, 'tmdbId': 155, 'plot': 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.', 'poster': '